# Gemini API: Audio Quickstart

This notebook provides an example of how to prompt Gemini Flash using an audio file. In this case, you'll use a [sound recording](https://www.jfklibrary.org/asset-viewer/archives/jfkwha-006) of President John F. Kennedy’s 1961 State of the Union address.


## Setup

### Install the Google GenAI SDK

Install the Google GenAI SDK from [npm](https://www.npmjs.com/package/@google/genai). 

```bash
$ npm install @google/genai
```

### Setup your API key

You can [create](https://aistudio.google.com/app/apikey) your API key using Google AI Studio with a single click.

Remember to treat your API key like a password. Don't accidentally save it in a notebook or source file you later commit to GitHub. In this notebook we will be storing the API key in a `.env` file. You can also set it as an environment variable or use a secret manager. 

Here's how to set it up in a `.env` file:

```bash
$ touch .env
$ echo "GEMINI_API_KEY=<YOUR_API_KEY>" >> .env
```

:::{.callout-tip}

Another option is to set the API key as an environment variable. You can do this in your terminal with the following command:

```bash
$ export GEMINI_API_KEY="<YOUR_API_KEY>"
```
:::

### Load the API key

To load the API key from the `.env` file, we will use the `dotenv` package. This package loads environment variables from a `.env` file into `process.env`. 

```bash
$ npm install dotenv
```

Then, we can load the API key in our code:


In [1]:
const dotenv = require("dotenv") as typeof import("dotenv");

dotenv.config({
  path: "../.env",
});

const GEMINI_API_KEY = process.env.GEMINI_API_KEY ?? "";
if (!GEMINI_API_KEY) {
  throw new Error("GEMINI_API_KEY is not set in the environment variables");
}
console.log("GEMINI_API_KEY is set in the environment variables");


GEMINI_API_KEY is set in the environment variables


:::{.callout-note}
In our particular case the `.env` is is one directory up from the notebook, hence we need to use `../` to go up one directory. If the `.env` file is in the same directory as the notebook, you can omit it altogether. 

```
│
├── .env
└── quickstarts
    └── Get_started_TTS.ipynb
```
:::


### Initialize SDK Client

With the new SDK, now you only need to initialize a client with you API key (or OAuth if using [Vertex AI](https://cloud.google.com/vertex-ai)). The model is now set in each call.


In [2]:
const google = require("@google/genai") as typeof import("@google/genai");

const ai = new google.GoogleGenAI({ apiKey: GEMINI_API_KEY });


### Select a model

Now select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](quickstarts/Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).


In [3]:
const tslab = require("tslab") as typeof import("tslab");

const MODEL_ID = "gemini-2.5-flash-preview-05-20";


## Upload an audio file with the File API

To use an audio file in your prompt, you must first upload it using the [File API](quickstarts/File_API.ipynb).


In [4]:
const fs = require("fs") as typeof import("fs");
const path = require("path") as typeof import("path");

const AUDIO_URL =
  "https://storage.googleapis.com/generativeai-downloads/data/State_of_the_Union_Address_30_January_1961.mp3";

const downloadFile = async (url: string, filePath: string) => {
  const response = await fetch(url);
  if (!response.ok) {
    throw new Error(`Failed to download image: ${response.statusText}`);
  }
  const buffer = await response.blob();
  const bufferData = Buffer.from(await buffer.arrayBuffer());
  fs.writeFileSync(filePath, bufferData);
};

const audioPath = path.join("../assets", "audio.mp3");

await downloadFile(AUDIO_URL, audioPath);


In [14]:
const audio_file = await ai.files.upload({
  file: audioPath,
  config: {
    displayName: "audio.mp3",
  },
});


## Use the file in your prompt


In [6]:
const audio_summary_response = await ai.models.generateContent({
  model: MODEL_ID,
  contents: [
    "Listen carefully to the following audio file. Provide a brief summary",
    google.createPartFromUri(audio_file.uri ?? "", audio_file.mimeType ?? ""),
  ],
});
tslab.display.markdown(audio_summary_response.text ?? "");


On January 30, 1961, President John F. Kennedy delivered his first State of the Union address, outlining significant challenges facing the United States. Domestically, he described an economy in recession with high unemployment, diminished growth, and rising prices. Internationally, he highlighted a widening balance of payments deficit and stressed the need to maintain the dollar's strength.

Kennedy then detailed a range of global crises, including Chinese Communist pressures in Asia, civil strife in the Congo, Soviet influence in Latin America (specifically Cuba), and disarray within NATO. He affirmed the U.S. commitment to freedom and independence worldwide but warned against underestimating adversaries.

The President proposed a comprehensive strategy to address these issues, focusing on strengthening military, economic, and diplomatic tools. This included modernizing defense capabilities (e.g., accelerating the Polaris submarine program), establishing a new foreign aid program ("Alliance for Progress" for Latin America, expanded Food for Peace), creating a National Peace Corps, and renewing efforts in arms control, scientific cooperation with the Soviet Union, and supporting the United Nations.

Kennedy concluded by calling for decisive action and accountability within the government, emphasizing that public service should be driven by initiative and responsibility. He acknowledged the difficult period ahead but expressed confidence in the American people's dedication to overcoming challenges and fulfilling the nation's promise for freedom and peace globally.

## Inline Audio

For small requests you can inline the audio data into the request, like you can with images. Use PyDub to trim the first 10s of the audio:


In [12]:
const ffmpeg = require("fluent-ffmpeg") as typeof import("fluent-ffmpeg");

const sliceAudio = async (audioBuffer: ArrayBuffer, durationMs: number): Promise<ArrayBuffer> =>
  new Promise((resolve, reject) => {
    const tempDir = path.resolve("../temp");
    if (!fs.existsSync(tempDir)) {
      fs.mkdirSync(tempDir);
    }

    const tempInputPath = path.join(tempDir, "temp_input.mp3");
    const tempOutputPath = path.join(tempDir, "temp_output.mp3");

    fs.writeFileSync(tempInputPath, Buffer.from(audioBuffer));

    ffmpeg(tempInputPath)
      .setStartTime(0)
      .setDuration(durationMs / 1000)
      .output(tempOutputPath)
      .on("end", () => {
        try {
          const outputBuffer = fs.readFileSync(tempOutputPath).buffer;

          // Cleanup temp files
          fs.unlinkSync(tempInputPath);
          fs.unlinkSync(tempOutputPath);

          resolve(outputBuffer);
        } catch (err) {
          reject(new Error(`Error reading output file: ${err.message}`));
        }
      })
      .on("error", (err) => {
        reject(new Error(`FFmpeg error: ${err.message}`));
      })
      .run();
  });

// slice the first 10 seconds of the audio file
const audioBuffer = fs.readFileSync(audioPath).buffer;
const slicedAudioBuffer = await sliceAudio(audioBuffer, 30000);
console.log("Sliced audio buffer length:", slicedAudioBuffer.byteLength);


Sliced audio buffer length: 480906


:::{.callout-note}

Note the following about providing audio as inline data:

- The maximum request size is 20 MB, which includes text prompts, system instructions, and files provided inline. If your file's size will make the total request size exceed 20 MB, then [use the File API](https://ai.google.dev/gemini-api/docs/audio?lang=javascript#upload-audio) to upload files.
- If you're using an audio sample multiple times, it is more efficient to use the [File API](https://ai.google.dev/gemini-api/docs/audio?lang=javascript#upload-audio).

:::


In [13]:
const inline_audio_response = await ai.models.generateContent({
  model: MODEL_ID,
  contents: [
    "Describe this audio clip",
    google.createPartFromBase64(Buffer.from(slicedAudioBuffer).toString("base64"), "audio/mp3"),
  ],
});
tslab.display.markdown(inline_audio_response.text ?? "");


The audio clip primarily features human speech by two distinct male voices.

1.  The first voice, an announcer or narrator, clearly states the context of the event: "The President's State of the Union address to a joint session of the Congress from the rostrum of the House of Representatives, Washington, D.C., January 30th, 1961."
2.  After a brief pause, the second voice, presumably the main speaker, begins addressing an audience with formal salutations like "Mr. Vice President" and "Members of the Congress," before stating, "It is a pleasure to return from whence I came."

Throughout the clip, a noticeable, low-frequency hum or buzzing sound is consistently present in the background. The overall audio quality suggests an archival recording with a somewhat dated feel, yet the speech remains clear.

## Get a transcript of the audio file

To get a transcript, just ask for it in the prompt. For example:


In [17]:
const transcript_response = await ai.models.generateContent({
  model: MODEL_ID,
  contents: [
    "Generate a transcript of the speech.",
    google.createPartFromUri(audio_file.uri ?? "", audio_file.mimeType ?? ""),
  ],
});
tslab.display.markdown(transcript_response.text ?? "");


### Refer to timestamps in the audio file

A prompt can specify timestamps of the form `MM:SS` to refer to particular sections in an audio file. For example:


In [18]:
const timestamped_transcript_response = await ai.models.generateContent({
  model: MODEL_ID,
  contents: [
    "Provide a transcript of the speech between the timestamps 02:30 and 03:29.",
    google.createPartFromUri(audio_file.uri ?? "", audio_file.mimeType ?? ""),
  ],
});
tslab.display.markdown(timestamped_transcript_response.text ?? "");


I speak today in an hour of national peril and national opportunity. Before my term has ended, we shall have to test anew whether a nation organized and governed such as ours can endure. The outcome is by no means certain. The answers are by no means clear. All of us together, this administration, this Congress, this nation, must forge those answers. But today, were I to offer after little more than a week in office, detailed legislation to remedy every national ill, the Congress would rightly wonder whether the desire for speed had replaced the duty of responsibility. My remarks, therefore, will be limited, but they will also be candid. To state the facts frankly is not to despair the future, nor to indict the past. The prudent heir takes careful inventory of his legacies and gives a faithful accounting to those whom he owes an obligation of trust. And while the occasion does not call for another recital of our blessings and assets, we do have no greater asset than the willingness of a free and determined people, through its elected officials, to face all problems frankly and meet all dangers free from panic or fear. The present state of our economy is disturbing. We take office in the wake of seven months of recession, three and one half years of slack, seven years of diminished economic growth, and nine years of falling farm income. Business bankruptcies have reached their highest level since the Great Depression. Since 1951, farm income has been squeezed down by 25%. Save for a brief period in 1958, insured unemployment is at the highest peak in our history. Of some five and one half million Americans who are without jobs, more than 1 million have been searching for work for more than four months. And during each month, some 150,000 workers are exhausting their already meager jobless benefit rights. Nearly 1/8 of those who are without jobs live almost without hope in nearly 100 specially distressed and troubled areas. The rest include new school graduates, unable to use their talents. Farmers forced to give up their part-time jobs which helped balance their family budgets. Skilled and unskilled workers laid off in such important industries as metals, machinery, automobiles, and apparel. Our recovery from the 1958 recession, moreover, was anemic and incomplete. Our gross national product never regained its full potential. Unemployment never returned to normal levels. Maximum use of our national industrial capacity was never fully restored. In short, the American economy is in trouble. The most resourceful industrialized country on Earth ranks among the last in the rate of economic growth. Since last spring, our economic growth rate has actually declined. Business investment is in a decline. Profits have fallen below predicted levels. Construction is off. A million unsold automobiles are in inventory. Fewer people are working, and the average work week has shrunk well below 40 hours. Yet prices have continued to rise, so that now too many Americans have less to spend for items that cost more to buy. Economic prophecy is at best an uncertain art. As demonstrated by the prediction one year ago from this same podium that 1960 would be, and I quote, "the most prosperous year in our history." Unquote. Nevertheless, forecast of continued slack and only slightly reduced unemployment through 1961 and 1962 have been made with alarming unanimity. And this administration does not intend to stand helplessly by. We cannot afford to waste idle hours and empty plant while awaiting the re-end of the recession. We must show the world what a free economy can do to reduce unemployment, to put unused capacity to work, to spur new productivity, and to foster high economic growth within a range of sound fiscal policies and relative price stability. I will propose to the Congress within the next 14 days measures to improve unemployment compensation through temporary increases in duration on a self-supporting basis, to provide more food for the families of the unemployed and to aid their needy children, to redevelop our areas of chronic labor surplus, to expand the services of the U.S. Employment Office, to stimulate housing and construction, to secure more purchasing power for our lowest paid workers by raising and expanding the minimum wage, to offer tax incentives for sound plant investment, to increase the development of our natural resources, to encourage price stability, and to take other steps aimed at ensuring a prompt recovery and paving the way for increased long-range growth. This is not a partisan program concentrating on our weaknesses. It is, I hope, a national program to realize our national strength. Efficient expansion at home, stimulating the new plant and technology that can make our goods more competitive is also the key to the international balance of payments problem. Laying aside all alarmist talk and panicky solutions, let us put that knotty problem in its proper perspective. It is true that since 1958, the gap between the dollars we spend or invest abroad and the dollars returned to us has substantially widened. This overall deficit in our balance of payments increased by nearly $11 billion in the last three years, and holders of dollars abroad converted them to gold in such a quantity as to cause a total outflow of nearly $5 billion of gold from our reserves. The 1959 deficit was caused in large part by our failure of our exports to penetrate foreign markets, the result both of restrictions on our goods and our own uncompetitive prices. The 1960 deficit, on the other hand, was more the result of an increase in private capital outflow, seeking new opportunity, higher return, or speculative advantage abroad. Meanwhile, this country has continued to bear more than its share of the West's military and foreign aid obligations. Under existing policies, another deficit of $2 billion is predicted for 1961, and individuals in those countries whose dollar position once depended on these deficits for improvement now wonder aloud whether our gold reserves will remain sufficient to meet our own obligations. All this is cause for concern, but it is not cause for panic, for our monetary and financial position remains exceedingly strong. Including our drawing rights in the International Monetary Fund and the gold reserve held as backing for our currency and Federal Reserve deposits, we have some $22 billion in total gold stocks and other international monetary reserves, and I now pledge that their full strength stands behind the value of the dollar for use if needed. Moreover, we hold large assets abroad. The total owed this nation far exceeds the claim upon our reserves, and our exports once again substantially exceed our imports. In short, we need not and we shall not take any action to increase the dollar price of gold from $35 an ounce, to impose exchange controls, to reduce our anti-recession efforts, to fall back on restrictive trade policies or to weaken our commitments around the world. This administration will not distort the value of the dollar in any fashion, and this is a commitment. Prudence and good sense do require, however, that new steps be taken to ease the payment deficits and prevent any gold crisis. Our success in world affairs has long depended in part upon foreign confidence in our ability to pay. A series of executive orders, legislative remedies, and cooperative efforts with our allies will get underway immediately. Aimed at attracting foreign investment and travel to this country, promoting American exports at stable prices and with more liberal government guarantees and financing, curbing tax and custom loopholes that encourage undue spending of private dollars abroad, and through OECD, NATO, and otherwise, sharing with our allies all efforts to provide for the common defense of the free world and the hopes for growth of the less developed lands. While the current deficit last, ways will be found to ease our dollar outlays abroad without placing the full burden on the families of men who have asked to serve our flag overseas. In short, whatever is required will be done to back up all our efforts abroad and to make certain that in the future as in the past, the dollar is as sound as a dollar. But more than our exchange of international payments is out of balance. The current federal budget for fiscal 1961 is almost certain to show a net deficit. The budget already submitted for fiscal 1962 will remain in balance only if the Congress enacts all the revenue measures requested, and only if an earlier and sharper upturn in the economy than my economic advisors now think likely produces the tax revenues estimated. Nevertheless, a new administration must of necessity build on the spending and revenue estimates already submitted. Within that framework, barring the development of urgent national defense needs or a worsening of the economy, it is my current intention to advocate a program of expenditures which, including revenue from a stimulation of the economy, will not of and by themselves unbalance the earlier budget. However, we will do what must be done. For our national household is clutted with unfinished and neglected tasks. Our cities are being engulfed in squalor. 12 long years after Congress declared our goal to be a decent home and a suitable environment for every American family, we still have 25 million Americans living in sub-standard homes. A new housing program, under a new Housing and Urban Affairs Department will be needed this year. Our classrooms contain 2 million more children than they can properly have room for, taught by 90,000 teachers not properly qualified to teach. One third of our most promising high school graduates are financially unable to continue the development of their talents. The war babies of the 1940s who overcrowded our schools in the 1950s are now descending in 1960 upon our colleges, with two college students for every one, 10 years from now. And our colleges are ill prepared. We lack the scientists, the engineers, and the teachers our world obligations require. We have neglected oceanography, saline water conversion, and the basic research that lies at the root of all progress. Federal grants for both higher and public school education can no longer be delayed. Medical research has achieved new wonders, but these wonders are too often beyond the reach of too many people, owing to a lack of income, particularly among the aged, a lack of hospital beds, a lack of nursing homes, and a lack of doctors and dentists. Measures to provide health care for the aged under Social Security and to increase the supply of both facilities and personnel must be undertaken this year. Our supply of clean water is dwindling. Organized and juvenile crime cost the taxpayers millions of dollars every year, making it essential that we have improved enforcement and new legislative safeguards. The denial of constitutional rights to some of our fellow Americans on account of race at the ballot box and elsewhere disturbs the national conscience and subjects us to the charge of world opinion that our democracy is not equal to the high promise of our heritage. Morality in private business has not been sufficiently spurred by morality in public business. A host of problems and projects in all 50 states, though not possible to include in this message, deserves and will receive the attention of both the Congress and the Executive Branch. On most of these matters, messages will be sent to the Congress within the next two weeks. But all these problems pale when placed besides those which confront us around the world. No man entering upon this office, regardless of his party, regardless of his previous service in Washington, could fail to be staggered upon learning even in this brief 10-day period, the harsh enormities of the trials through which we must pass in the next four years. Each day the crises multiply. Each day their solution grows more difficult. Each day we draw nearer the hour of maximum danger as weapons spread and hostile forces grow stronger. I feel I must inform the Congress that our analysis over the last 10 days make it clear that in each of the principal areas of crisis, the tide of events has been running out and time has not been our friend. In Asia, the relentless pressures of the Chinese Communists menace the security of the entire area from the borders of India and South Vietnam to the jungles of Laos, struggling to protect its newly won independence. We seek in Laos what we seek in all Asia, and indeed in all of the world, freedom for the people and independence for the government, and this nation shall persevere in our pursuit of these objectives. In Africa, the Congo has been brutally torn by civil strife, political unrest, and public disorder. We shall continue to support the heroic efforts of the United Nations to restore peace and order, efforts which are now endangered by mounting tensions, unsolved problems, and decreasing support from many member states. In Latin America, communist agents seeking to exploit that region's peaceful revolution of hope have established a base on Cuba, only 90 miles from our shore. Our objection with Cuba is not over the people's drive for a better life. Our objection is to their domination by foreign and domestic tyranny. Cuban social and economic reforms should be encouraged. Questions of economics and trade policy can always be negotiated, but communist domination in this hemisphere can never be negotiated. We are pledged to work with our sister Republics to free the Americas of all such foreign domination and all tyranny, working towards the goal of a free hemisphere of free governments, extending from Cape Horn to the Arctic Circle. In Europe, our alliances are unfiled and in some disarray. The unity of NATO has been weakened by economic rivalry and partially eroded by national interest. It has not yet fully mobilized its resources nor fully achieved a common outlook. Yet no Atlantic power can meet on its own the mutual problems now facing us in defense, foreign aid, monetary reserves, and a host of other areas, and our close ties with those whose hopes and interests we share are among this nation's most powerful assets. Our greatest challenge is still the world that lies beyond the Cold War. But the first great obstacle is still our relations with the Soviet Union and communist China. We must never be loud into believing that either power has yielded its ambitions for world domination, ambitions which they forcefully restated only a short time ago. On the contrary, our task is to convince them that aggression and subversion will not be profitable roots to pursue these ends. Open and peaceful competition for prestige, for market, for scientific achievement, even for men's minds is something else again. For if freedom and communism would compete for man's allegiance in a world at peace, I would look to the future with ever increasing confidence. To meet this array of challenges, to fulfill the role we cannot avoid on the world scene, we must re-examine and revise our whole arsenal of tools. One must not overshadow the other. On the presidential coat of arms, the American Eagle holds in his right talent the olive branch, while in his left he holds a bundle of arrows. We intend to give equal attention to both. First, we must strengthen our military tools. We are moving into a period of uncertain risk and great commitment in which both the military and diplomatic possibilities require a free world force so powerful as to make any aggression clearly futile. Yet in the past, lack of a consistent, coherent military strategy, the absence of basic assumptions about our national requirements, and the faulty estimate and duplication arising from inner service rivalries have all made it difficult to assess accurately how adequate or inadequate our defenses really are. I have therefore instructed the Secretary of Defense to reappraise our entire defense strategy. Our ability to fulfill our commitments, the effectiveness, vulnerability, and dispersal of our strategic basis, forces, and warning systems. The efficiency and economy of our operation and organization, the elimination of obsolete basis and installations, and the adequacy, modernization, and mobility of our present conventional and nuclear forces and weapon systems in the light of present and future dangers. I have asked for preliminary conclusions by the end of February, and I then shall recommend whatever legislative, budgetary, or executive action is needed in the light of these conclusions. In the meantime, I have asked the Defense Secretary to initiate immediately three steps most clearly needed now. First, I have directed prompt attention to increase our airlift capacity. Obtaining additional air transport mobility and obtaining it now will better assure the ability of our conventional forces to respond without discrimination and speed, with discrimination and speed, to any problem at any spot on the globe at any moment's notice. In particular, it will enable us to meet any deliberate effort to avoid or divert our forces by starting limited wars in widely scattered parts of the globe. B, I have directed prompt action to step up our Polaris submarine program. Using using unobligated shipbuilding funds now to let contracts originally scheduled for the next fiscal year, will build and place on station at least nine months earlier than planned substantially more units of a crucial deterrent. A fleet that will never attack first, but possess sufficient powers of retaliation concealed beneath the sea to discourage any aggressor from launching an attack upon our security. C, I have directed prompt action to accelerate our entire missile program. Until the Secretary of Defense's reappraisal is completed, the emphasis here will be largely on improved organization and decision making, on cutting down the wasteful duplications and the time lag that have handicapped our whole family of missiles. If we are to keep the peace, we need an invulnerable missile force, powerful enough to deter any aggressor from even threatening an attack that he would know could not destroy enough of our own forces to prevent his own destruction. For as I said upon taking the oath of office, only when our arms are sufficient beyond doubt, can we be certain beyond doubt that they will never be employed. Secondly, we must improve our economic tools. Our role is essential and unavoidable in the construction of a sound and expanding economy for the entire non-communist world, helping other nations build the strength to meet their own problems, to satisfy their own aspirations, to surmount their own dangers. The problems in achieving this goal are towering and unprecedented. The response must be towering and unprecedented as well. Much as Lend Lease and the Marshall Plan were in earlier years, which brought such fruitful results. I intend to ask the Congress for authority to establish a new and more effective program for assisting the economic, educational, and social development of other countries and continents. That program must stimulate and take more effectively into account the contributions of our allies and provide central policy direction for all our own programs that now so often overlap, conflict, or diffuse our energies and resources. Such a program compared to past programs will require more flexibility for short run emergencies, more commitment to long-term development, new attention to education at all levels, greater emphasis on the recipient nation's role, their effort, their purpose, with greater social justice for their people, a broader distribution and participation by their people, and more efficient public administration and more efficient tax systems of their own. And orderly planning for national and regional development instead of a piecemeal approach. I hope the Senate will take early action approving the convention establishing the organization for economic cooperation and development. This will be an important instrument in sharing with our allies this development effort, working towards the time when each nation will contribute in proportion to its ability to pay. For while we are prepared to assume our full share of these huge burdens, we cannot and must not be expected to bear them alone. To our sister Republics of the South, we have pledged a new alliance for progress, Alianza para Progreso. Our goal is a free and prosperous Latin America, realizing for all its states and all its citizens a degree of economic and social progress that matches their historic contributions of culture, intellect, and liberty. To start this nation's role at this time in that alliance of neighbors, I am recommending the following. That the Congress appropriate in full the $500 million fund pledged by the Act of Bogota to be used not as an instrument of the Cold War, but as a first step in the sound development of the Americas. That a new interdepartmental task force be established under the leadership of the Department of State to coordinate at the highest level all policies and programs of concern to the Americas. That our delegates to the OAS, working with those of other members, strengthen that body as an instrument to preserve the peace and to prevent foreign domination anywhere in the hemisphere. That in cooperation with other nations, we launch a new hemispheric attack on illiteracy and inadequate educational opportunities at all levels. And finally, that a food for peace mission be sent immediately to Latin America to explore ways in which our vast food abundance can be used to help hunger and malnutrition in certain areas of suffering in our own hemisphere. This administration is expanding its Food for Peace program in every possible way. The product of our abundance must be used more effectively to relieve hunger and help economic growth in all corners of the globe. And I have asked the director of this program to recommend additional ways in which these surpluses can advance the interests of world peace, including the establishment of world food reserves. An even more valuable national asset is our reservoir of dedicated men and women, not only at our college campuses, but in every age group, who have indicated their desire to contribute their skills, their efforts, and a part of their lives to the fight for world order. We can mobilize this talent through the formation of a national Peace Corps, enlisting the services of all those with desire and capacity to help foreign lands meet their urgent needs for trained personnel. Finally, while our attention is centered on the development of the non-communist world, we must never forget our hopes for the ultimate freedom and welfare of the Eastern European people. In order to be prepared to help reestablish historic ties of friendship, I am asking the Congress for increased discretion to use economic tools in this area, whenever this is found to be clearly in the national interest. This will require amendment of the Mutual Defense Assistance Control Act, along the lines I proposed as a member of the Senate, and upon which the Senate voted last summer. Meanwhile, I hope to explore with the Polish government the possibility of using our frozen Polish funds on projects of peace that will demonstrate our abiding friendship and interest in the people of Poland. Third, we must sharpen our political and diplomatic tools. The means of cooperation and agreement on which an enforceable world order must ultimately rest. I have already taken steps to coordinate and expand our disarmament effort, to increase our programs of research and study, and to make arms control a central goal of our national policy under my direction. The deadly arms race and the huge resources it absorbs have too long overshadowed all else we must do. We must prevent that arms racing from spreading to new nations, to new nuclear powers, powers and to the outer reaches of space. We must make certain that our negotiators are better informed and better prepared to formulate workable proposals of our own and to make sound judgment about the proposals of others. I have asked the other governments concerned to agree to a reasonable delay in the talks on a nuclear test ban, and it is our intention to resume negotiations prepared to reach a final agreement with any nation that is equally willing to agree to an effective and enforceable treaty. We must increase our support of the United Nations as an instrument to end the Cold War instead of an arena in which to fight it. In recognition of its increasing importance and the doubling of its membership, we are enlarging and strengthening our own mission to the UN. We shall help ensure that it is properly financed. We shall work to see that the integrity of the office of the Secretary General is maintained. And I would address a special plea to the smaller nations of the world to join with us in strengthening this organization which is far more essential to their security than it is to ours. The only body in the world where no nation need be powerful to be secure, where every nation has an equal voice, and where any nation can exert influence not according to the strength of its armies, but according to the strength of its ideas. It deserves the support of all. Finally, this administration intends to explore promptly all possible areas of cooperation with the Soviet Union and other nations to invoke the wonders of science instead of its terrors. Specifically, I now invite all nations, including the Soviet Union, to join with us in developing a weather prediction program, in a new communications satellite program, and in preparation for probing the distant planets of Mars and Venus, probes which may someday unlock the deepest secrets of the universe. Today this country is ahead in the science and technology of space, while the Soviet Union is ahead in the capacity while the Soviet Union is ahead in the capacity to lift large vehicles into orbit. Both nations would help themselves as well as other nations by removing these endeavors from the bitter and wasteful competition of the Cold War. The United States would be willing to join with the Soviet Union and the scientists of all nations in a greater effort to make the fruits of this new knowledge available to all. And beyond that, in an effort to extend farm technology to hungry nations, to wipe out disease, to increase the exchanges of scientists and knowledge, and to make our own laboratories available to technicians of other lands who lack the facilities to pursue their own work. Where nature makes natural allies of us all. We can demonstrate that beneficial relations are possible even with those with whom we most deeply disagree. And this must someday be the basis of world peace and world law. I have commented on the state of the domestic economy, our balance of payments, our federal and social budget, and the state of the world. I would like to conclude with a few remarks about the state of the Executive Branch. We have found it full of honest and useful public servants. But their capacity to act decisively at the exact time action is needed has too often been muffled in the morass of committees, timidities, and fictitious theories, which have created a growing gap between decision and execution, between planning and reality. In a time of rapidly deteriorating situations at home and abroad, this is bad for the public service and particularly bad for the country. And we mean to make a change. I have pledged myself and my colleagues in the Cabinet to a continuous encouragement of initiative, responsibility, and energy in serving the public interest. Let every public servant know whether his post is high or low, that a man's rank and reputation in this administration will be determined by the size of the job he does and not by the size of his staff, his office, or his budget. Let it be clear that this administration recognizes the value of dissent and daring, that we greet healthy controversy as the hallmark of healthy change. Let the public service be a proud and lively career. And let every man and woman who works in any area of our national government, in any branch, at any level, be able to say with pride and with honor in future years, I served the United States Government in that hour of our nation's need. For only with complete dedication by us all to the national interest, can we bring our country through the troubled years that lie ahead. Our problems are critical, the tide is unfavorable, the news will be worse before it is better. And while hoping and working for the best, we should prepare ourselves now for the worst. We cannot escape our dangers. Neither must we let them drive us into panic or narrow isolation. In many areas of the world, where the balance of power already rests with our adversaries, the forces of freedom are sharply divided. It is one of the ironies of our time that the techniques of a harsh and repressive system should be able to instill discipline and order in its servants, while the blessings of liberty have too often stood for privilege, materialism, and a life of ease. But I have a different view of liberty. Life in 1961 will not be easy. Wishing it, predicting it, even asking for it will not make it so. There will be further setbacks before the tide is turned, but turn it we must. The hopes of all mankind rest upon us. Not simply upon those of us in this chamber, but upon the peasant in Laos, the fisherman in Nigeria, the exile from Cuba, the spirit that moves every man and nation who shares our hopes for freedom and the future. And in the final analysis, they rest most of all upon the pride and perseverance of our fellow citizens of the Great Republic. In the words of a great president whose birthday we honor today, closing his final State of the Union message, 16 years ago, we pray that we may be worthy of the unlimited opportunities that God has given us.

## Use a Youtube video


In [20]:
const YOUTUBE_URL = "https://www.youtube.com/watch?v=RDOMKIw1aF4";

const youtube_response = await ai.models.generateContent({
  model: MODEL_ID,
  contents: [
    `
        Analyze the following YouTube video content. Provide a concise summary covering:

    1.  **Main Thesis/Claim:** What is the central point the creator is making?
    2.  **Key Topics:** List the main subjects discussed, referencing specific examples or technologies mentioned (e.g., AI models, programming languages, projects).
    3.  **Call to Action:** Identify any explicit requests made to the viewer.
    4.  **Summary:** Provide a concise summary of the video content.

    Use the provided title, chapter timestamps/descriptions, and description text for your analysis.
        `,
    google.createPartFromUri(YOUTUBE_URL, "video/x-youtube"),
  ],
});
tslab.display.markdown(youtube_response.text ?? "");


Here's an analysis of the YouTube video content:

    1.  **Main Thesis/Claim:** Google's newly released Gemini 2.5 Pro is a highly capable AI for coding, demonstrating impressive abilities in code generation and refactoring across various programming languages, often outperforming or competing strongly with other leading AI models. While it excels at functional code, it still faces challenges in accurately translating complex visual mockups into fully functional and aesthetically precise UIs.

    2.  **Key Topics:**
        *   **Gemini 2.5 Pro Capabilities:** Highlighted for its large 1M token context window and overall performance as a coding AI.
        *   **Code Generation:**
            *   Successfully generated an "Ultimate Tic-Tac-Toe" game in Java (Swing) in a single attempt ("one-shot").
            *   Created a "Kitten Cannon" style game using P5.js, requiring minor error corrections (fixed with two additional prompts, making it a "three-shot" overall).
            *   Recreated a static HTML/CSS layout of the X (Twitter) website, demonstrating visual accuracy but no dynamic functionality.
        *   **Code Refactoring:** Showcased a significant improvement in refactoring complex Rust code, converting traditional loops to more idiomatic iterators and generally cleaning up the codebase, surpassing other models in this task.
        *   **UI Generation Limitations:** Demonstrated struggles with building a precise landing page using Vite, React, and Tailwind CSS from a visual mockup, indicating difficulties with complex visual translation and asset integration.
        *   **Benchmarking and Performance:** Compared Gemini 2.5 Pro's performance against other prominent AI models (OpenAI's GPT-4/o3-mini, Claude 3, DeepSeek, Grok) across reasoning, science, mathematics, code generation, and long context. Gemini showed strong leads in reasoning, science, and mathematics, and competitive performance in code generation and editing.
        *   **Information Currentness (Grounding):** Noted Gemini's training data extends to March 2025 and its ability to "ground" responses with Google Search for real-time information (e.g., current React.js version).

    3.  **Call to Action:**
        *   Encourages viewers to share their experiences with Gemini 2.5 Pro in the comments.
        *   Asks Rust developers for their opinion on the refactored Rust code.
        *   Requests viewers to subscribe, like the video, and enable notifications.

    4.  **Summary:** The video showcases the impressive coding prowess of Google's new Gemini 2.5 Pro AI model, emphasizing its large context window and strong capabilities in generating and refactoring code. The creator successfully uses Gemini to build a Java Tic-Tac-Toe game in one attempt, a P5.js game with minor fixes, and a static Twitter UI. A key highlight is Gemini's superior ability to refactor complex Rust code into cleaner, more idiomatic forms. While the model excels in functional coding, it encounters limitations when tasked with generating complex, pixel-perfect user interfaces directly from visual mockups. The creator also presents benchmark comparisons, demonstrating Gemini's competitive edge over other leading AI models in various domains, including its up-to-date knowledge base. The video concludes by recommending Gemini 2.5 Pro as a powerful and potentially cost-effective tool for developers.

## Count audio tokens

You can count the number of tokens in your audio file using the [countTokens](https://googleapis.github.io/js-genai/release_docs/classes/models.Models.html#counttokens) method.

Audio files have a fixed per second token rate (more details in the dedicated [count token quickstart](quickstarts/Counting_Tokens.ipynb).)


In [21]:
const count_tokens_response = await ai.models.countTokens({
  model: MODEL_ID,
  contents: [google.createPartFromUri(audio_file.uri ?? "", audio_file.mimeType ?? "")],
});
tslab.display.markdown(`Token count for the audio file: \`${count_tokens_response.totalTokens}\``);


Token count for the audio file: `83528`

## Next Steps

### Useful API references:

More details about Gemini API's [vision capabilities](https://ai.google.dev/gemini-api/docs/vision) in the documentation.

If you want to know about the File API, check its [API reference](https://ai.google.dev/api/files) or the [File API](quickstarts/File_API.ipynb) quickstart.

### Related examples

Check this example using the audio files to give you more ideas on what the gemini API can do with them:

- Share [Voice memos](examples/Voice_memos.ipynb) with Gemini API and brainstorm ideas.

### Continue your discovery of the Gemini API

Have a look at the [Audio](quickstarts/Audio.ipynb) quickstart to learn about another type of media file, then learn more about [prompting with media files](https://ai.google.dev/tutorials/prompting_with_media) in the docs, including the supported formats and maximum length for audio files.
